# ShakespearGPT
### Dans ce notebook, je vais essayer de reproduire le ShakespearGPT proposé par Andrej Karpathy dans sa vidéo YouTube. Je fais cela dans un but éducatif, pour comprendre le fonctionnement des LLMs.




## I - Importation et préparation des données

C'est Andrej Karpathy qui le dit, on commence toujours par importer les données. Ici, on importe le contenu de TinyShakespear, qui est une concaténation de l'ensemble des écrits de Shakespeare.

J'importe son contenu dans ce notebook dans le fichier input.txt que je récupère sur le repo github du projet.

Je l'importe ci-dessous dans la variable string text.

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
input_file = open('input.txt', 'r', encoding='utf-8')
text = input_file.read()

--2025-10-20 07:00:58--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2025-10-20 07:00:58 (22.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



Maintenant qu'on a nos données, comme tout bon ingénieur, on va s'intéresser un peu à ce qu'on va manipuler avant d'aller plus loin.

In [2]:
print("Le texte contient", len(text), "caractères.")
print("Le texte contient", len(set(text)), "caractères uniques.")
print("\nLes 500 premiers caractères du texte sont :\n", text[:500])
print("\nLes caractères de 24314 à 24414 du texte sont :\n", text[24314:24414])

Le texte contient 1115394 caractères.
Le texte contient 65 caractères uniques.

Les 500 premiers caractères du texte sont :
 First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor

Les caractères de 24314 à 24414 du texte sont :
 lish in our stands,
Nor cowardly in retire: believe me, sirs,
We shall be charged again. Whiles we h


Ok, on a un peu plus d'informations sur le texte, et surtout on sait qu'on n'a pas importé n'importe quoi : ça ressemble bien à du shakespear.
Une information importante qu'on a maintenant, c'est le nombre de caractère différents dans le texte : 65.
Notre GPT devra choisir, à chaque itération, un caractère parmi 65.

Étudions un peu ces caractères.

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Toutes les lettres en majuscule, toutes les lettres en minuscule, espace, saut à la ligne, dollars (??), point etc.

Maintenant qu'on a bien étudié nos données, qu'on s'est fait une plutôt bonne idée de ce qui les constituait, on va passer à la première étape importante : l'encodage et le décodage en tokens.

Un token, c'est un morceau de texte que notre GPT peut prédire à chaque itération. Pour faire simple, c'est une unité de texte. Pourquoi est-ce qu'on a besoin des tokens ? Ça permet de transformer le problème de génération de texte en problème de génération d'entier. En plus, on peut choisir nos tokens de différentes manières, certaines plus efficaces que d'autres.

Dans notre cas, on va pas se prendre trop la tête, on a un plutôt bon candidat : les caractères. Chaque caractère sera un token. Pour les encoder, il suffit donc d'associer à chaque caractère un entier.

Programmons l'encodage et le décodage, pour obtenir les tokens qu'on utilisera pour ce projet.

In [4]:
str_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_str = { i:ch for i,ch in enumerate(chars) }

# L'encodage ci-dessous suppose que l'entrée ne contient que les caractères de notre texte
def encode(s) :
  res = []
  for i in range(len(s)) :
    res.append(str_to_int[s[i]])
  return res

def decode(s) :
  res = ""
  for i in range(len(s)) :
    res += chars[s[i]]
  return res

print(encode("Ceci est un texte que je vais encoder puis decoder."))
print(decode(encode("Ceci est un texte que je vais encoder puis decoder.")))

[15, 43, 41, 47, 1, 43, 57, 58, 1, 59, 52, 1, 58, 43, 62, 58, 43, 1, 55, 59, 43, 1, 48, 43, 1, 60, 39, 47, 57, 1, 43, 52, 41, 53, 42, 43, 56, 1, 54, 59, 47, 57, 1, 42, 43, 41, 53, 42, 43, 56, 8]
Ceci est un texte que je vais encoder puis decoder.


Sur cet exemple, on comprend un peu pourquoi notre encodage n'est pas optimal. "est" par exemple est une séquence qui apparaîtra très fréquemment dans un texte en français, ça serait donc pratique d'avoir un token dédier. En choisissant bien les tokens, on peut réduire la taille de l'encodage. Il faut en réalité trouver le bon équilibre entre nombre de tokens et taille des encodages. L'encodage de GPT2, par exemple, comptait plus de 50000 tokens.

Ceci dit, notre encodage, assez simple à comprendre, fera l'affaire pour ce projet.

Maintenant qu'on a défini tokens et encodage, appliquons tout ça à notre texte.

In [5]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

En analysant un peu ce qui apparaît, on remarque d'abord que on a bien autant d'entier dans data que de caractères dans text, puis que notre encodage a l'air d'avoir bien fonctionné, si l'on compare les premiers entiers aux premiers caractères.

On a nos données, prêtes à entraîner notre modèle... À un détail prêt : pour s'assurer que le modèle fonctionne toujours bien sur des nouvelles données, qu'il n'est pas seulement bon à reproduire ce qu'on lui a donné pour s'entraîner, on va séparer nos données en deux ensemble : le training set et le validation set

In [6]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

Voilà, nos données sont séparées en un ensemble d'entraînement, et un ensemble de validation.

Maintenant, on ne va pas donner un texte entier et demander de prédire le caractère suivant. On va donner à notre algorithme des blocks de texte encodé, et lui demander de prédire, selon ce petit contexte, le caractère suivant.

Ici, on va considérer des blocks de 8 tokens.

In [7]:
block_size = 8

print(train_data[:block_size+1])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


Ça, c'est un block d'entrée et le caractère qui suit.
Mais si on fait l'effort de regarder entre les caractères, on s'aperçoit que ce block de taille block_size en contient en réalité 8.

En effet,on va entraîner notre modèle sur des entrées de taille block_size, mais on veut aussi qu'il soit capable de faire une prédiction en se basant uniquement sur un contexte d'un seul caractère.

Ainsi, un block de 8 tokens nous donne en réalité 8 prédictions à faire pour s'entraîner.

In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size) :
  context = x[:t+1]
  target = y[t]
  print(f"Lorsque le contexte est {context}, la cible est {target}")

Lorsque le contexte est tensor([18]), la cible est 47
Lorsque le contexte est tensor([18, 47]), la cible est 56
Lorsque le contexte est tensor([18, 47, 56]), la cible est 57
Lorsque le contexte est tensor([18, 47, 56, 57]), la cible est 58
Lorsque le contexte est tensor([18, 47, 56, 57, 58]), la cible est 1
Lorsque le contexte est tensor([18, 47, 56, 57, 58,  1]), la cible est 15
Lorsque le contexte est tensor([18, 47, 56, 57, 58,  1, 15]), la cible est 47
Lorsque le contexte est tensor([18, 47, 56, 57, 58,  1, 15, 47]), la cible est 58


On commence à se faire une bonne idée de ce qu'on va donner en entrée à notre modèle. La dernière chose à faire, c'est de rassembler plusieurs entrées en **batch**.

Entraîner sur une donnée, ça prend un certain temps. Or, pendant qu'on entraîne sur une donnée, il nous reste de la place pour en entraîner d'autres en parallèle.

On ne va donc pas donner nos entrées une par une, mais les rassembler en **batch**, qui désigne un ensemble d'entrées mises en parallèle.

Il reste donc à définir ces batchs, après quoi nous auront nos données prêtes pour entraîner notre modèle.

In [9]:
torch.manual_seed(1337)
batch_size = 4

def get_batch(training) :
  data = train_data if training else val_data
  random_sample_index = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in random_sample_index]) #torch.stack concatenates tensors given in one tensor of higher dimension
  y = torch.stack([data[i+1:i+block_size+1] for i in random_sample_index])
  return x, y

xb, yb = get_batch(training = True)
print("inputs :")
print(xb.shape)
print(xb)
print("targets :")
print(yb.shape)
print(yb)

print("\n\n")

for b in range(batch_size) :
  for t in range(block_size) :
    context = xb[b, :t+1]
    target = yb[b, t]
    print("Quand l'entrée est", context, "la prédiction attendue est :", target)

inputs :
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets :
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])



Quand l'entrée est tensor([24]) la prédiction attendue est : tensor(43)
Quand l'entrée est tensor([24, 43]) la prédiction attendue est : tensor(58)
Quand l'entrée est tensor([24, 43, 58]) la prédiction attendue est : tensor(5)
Quand l'entrée est tensor([24, 43, 58,  5]) la prédiction attendue est : tensor(57)
Quand l'entrée est tensor([24, 43, 58,  5, 57]) la prédiction attendue est : tensor(1)
Quand l'entrée est tensor([24, 43, 58,  5, 57,  1]) la prédiction attendue est : tensor(46)
Quand l'entrée est tensor([24, 43, 58,  5, 57,  1, 46]) la prédiction attendue est : tensor(43)
Quand l'entrée e

## II - Construction d'un premier modèle BLM

Nos données sont prêtes, au bon format pour notre transformer!

On peut maintenant s'intéresser au modèle que l'on va implémenter.

Pour commencer cela, on va partir de la base des NLP : les Bigram Language Model. Un BLM, c'est un modèle dont l'unique rôle est de prendre un token, et de prédire celui qui suit. Il y a plusieurs manières de le faire. On peut prendre l'ensemble des données, regarder la fréquence d'apparition de chaque couple de token possible, en déduire la prédiction la plus probable pour le token suivant. On peut aussi construire un réseau neuronal qui va se charger de faire la prédiction, et c'est ce qu'on va faire ici.

1) Forward

Étant donné un batch xb en entrée et les targets yb correspondantes, notre modele va construire une **embedding table de dimension batch_size * block_size * num_classes** selon xb, qui donnera, pour chaque entrée du batch, les scores (~probas) de chaque token pour toutes les sous-entrées. Un peu compliqué tout ça.
Avec cette embedding table, le modèle fera sa prédiction, et on calcule ensuite a perte avec cross_entropy. Pour faire cela, on ajuste les dimensions de notre prédiction et les targets (pour coller aux entrées attendues par la fonction torch.functional.cross_entropy).

2) Generate

On passe à l'étape plus fun, le résultat voulu : la génération de texte. Pour cette fonction, on répète juste autant de fois qu'on veut la prédiction du prochain caractère basé uniquement sur le caractère précédent. On garde la prédiction qui a la probabilité la plus élevée, on l'ajoute à l'entrée, et on répète le procédé sur notre entrée agrandie.

In [10]:
import torch.nn as nn
from torch.nn import functional as F

batch_size = 32
block_size = 8
vocab_size = len(chars)
n_embd = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class FirstBiagramLanguageModel(nn.Module) :
  def __init__(self) :
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # matrice de taille vocab_size * n_embd
    self.position_embedding_table = nn.Embedding(block_size, n_embd) # matrice de taille block_size * n_embd
    self.lm_head = nn.Linear(n_embd, vocab_size) # matrice de taille n_embd * vocab_size, qui permettra de transformer token_embd en logits

  def forward(self, idx, targets=None) :
    B, T = idx.shape

    tok_emb = self.token_embedding_table(idx) # idx est de dimensions B * T, tok_emb est donc de dimensions B * T * C (où C = n_embd)
    pos_emb = self.position_embedding_table(torch.arange(T, device = device)) # dimensions T * C, ajoute l'information de la position du token dans le block
    x = tok_emb + pos_emb # dimensions B * T * C
    logits = self.lm_head(x) # idx est de dimensions B * T, logits est donc de dimensions B * T * vocab_size

    if targets == None :
      loss = None
    else :
      B, T, C = logits.shape
      logits = logits.view(B*T, C) # on redimensionne nos prédictions pour calculer la loss
      targets = targets.view(B*T)

      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_tokens_generated) :
    for i in range(max_tokens_generated) :
      idx_cond = idx[:, -block_size:]  # tronquer à la longueur maximale autorisée
      logits, loss = self.forward(idx_cond)
      logits = logits[:, -1, :] # on isole les prédictions pour le prochain caractère basé sur celui qui précède
      probs = F.softmax(logits, dim=-1) # on calcule les probas
      idx_next = torch.multinomial(probs, num_samples=1) # on sélectionne le caractère suivant selon les probas obtenues
      idx = torch.cat((idx, idx_next), dim=1) # on ajoute le caractère suivant à notre entrée
    return idx

m = FirstBiagramLanguageModel()
logits, loss = m(xb, yb)

print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long) #on donne en premier le caractère de saut de ligne
print(decode(m.generate(idx, max_tokens_generated=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.6424, grad_fn=<NllLossBackward0>)

:RTbVTkMTUwF C$?3fHvOvsmEEDDoys!SZgyGrRX:DdqBsmroU&SjrPr:EjT!hjmfHDHd3cOx.vvgvuvL&egm-CvLif.z Ur3RmC


On a un modèle bien construit, et une belle fonction de génération qui est déjà construite de sorte à pouvoir, plus tard, prendre en compte le contexte, et pas seulement le dernier caractère !

Par contre, le résultat n'est pas fou.. En même temps, on le génère complétement aléatoirement. On va entraîner notre modèle pour voir s'il peut s'améliorer.

On va utiliser le AdamW optimizer de pytorch, et lancer une boucle d'optimisation plutôt classique : on prend prend un batch aléatoire, on fait une prédiction, on calcule la loss, on calcule les gradients, puis on optimise en conséquence

On va voir jusqu'ou on peut descendre en loss avec cette optimisation.

In [11]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
iterations = 10000

for i in range(iterations) :
  xb, yb = get_batch(training = True)
  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())


2.384690761566162


En répétant 1000 itérations, on descend à une perte de 3.7, en répétant l'optimisation encore 10000 fois on descend jusqu'à 2.56, puis encore 10000 itérations pour descendre à une perte de 2.41.

Ok on voit que la perte a bien descendu, donc on va réessayer de générer du texte. Ça ne sera probablement pas du Shakespear, mais sûrement un peu plus compréhensible que ce qu'on avait généré sans entraînement.

In [12]:
idx = torch.zeros((1, 1), dtype=torch.long) #on donne en premier le caractère de saut de ligne
print(decode(m.generate(idx, max_tokens_generated=300)[0].tolist()))



ARKI it?
Angirounesoacat the Fat haceseliss me--s t ese we ds

NCHULI:
OMet monce : Stus me h! al-p blllkscet pe lar ce, haw, d our, ly be d p, beleer
ANCande my innofoothesound is'lf co o he sottcothealin athe ot bande s wnetourmapuseas ated a t.
O:

T:
GLLINGLARotind ile hingoue arilo se
AUSewo g


On ne va pas attirer beaucoup de monde dans les théâtres avec un texte comme ça, mais on commence à reconnaître la forme d'une pièce de théâtre.

Le problème, c'est qu'on ne fait nos prédictions que sur le dernier caractère. Pour obtenir un résultat satisfaisant, il va falloir commencer à prendre en compte le contexte.

C'est là que ça devient intéressant.

## III - Introduction mathématique à la self-attention

Avant d'avancer, on va essayer de bien comprendre ce qui se passe derrière le principe de self-attention. Comprendre l'idée mathématique est la clé d'une bonne implémentation. On va faire un exemple pour illustrer tout ça.

In [13]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

On a construit une entrée aléatoire (ce n'est pas l'entrée qui importe, mais ce qu'on va en faire).

L'idée est la suivante, pour le caractère à l'instant t, on veut conserver une information des caractères à l'instant t-1, t-2 etc. Ainsi, à tout instant, le caractère conserve une empreinte de ceux qui le précèdent.

Comment conserver l'information ? Il y a plein de manière de le faire, mais on va faire au plus simple pour l'instant : on va faire la moyenne des caractères qui précèdent et du caractère actuel pour obtenir notre nouveau caractère.

In [14]:
xbow = torch.zeros((B,T,C)) # bow stands for bag of words -> on fait juste la moyenne d'un groupe de mots (ici des caractères)
for b in range(B) :
  for t in range(T) :
    xprev = x[b, :t+1]
    xbow[b, t] = torch.mean(xprev, 0)

Le code ci-dessus nous donne xbow qui est le résultat de l'opération qu'on voulait faire. Seulement, on fait notre calcul d'une manière pas très efficace.

On va voir comment rendre ce calcul efficace par le calcul marticiel.

In [15]:
torch.manual_seed(42)

a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print("tril :")
print(a)
print("\n")
print("b :")
print(b)
print("\n")
print("c :")
print(c)

tril :
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])


b :
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])


c :
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


Avec ce calcul matriciel, et plus précisément, avec cette matrice a, on peut faire le calcul de xbow (ici c) étant donné x (ici b) de manière efficace.

In [16]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # Les dimensions (T, T) et (B, T, C) ne permettent pas une multiplication matricielle,
                # l'opérateur @ crée une dimension B, donc l'opération est appliquée à toutes les entrée du batch,
                # le tout en parallèle donc plus efficacement

On peut également réaliser cette opération, obtenir la même matrice wei avec un softmax (en l'appliquant à la matrice tril ou les 1 deviennent 0 et les 0 deviennent -inf). C'est cette méthode là que l'on va retenir pour la suite.

In [17]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x

Ce qu'il faut retenir de tout ça, c'est qu'on peut réécrire une entrée pour que chaque caractère représente aussi le contexte dans lequel il apparaît, à l'aide d'une simple multiplication matricielle. La matrice wei définit les poids, et donc l'importance, l'affinité, de chaque caractère passés.

## IV - Self-Attention

On arrive enfin au principe clé de notre algorithme : la self-attention.

On a vu juste avant qu'on pouvait garder les informations des caractères passés, mais on l'a fait pour l'instant d'une manière assez simple, juste avec la moyenne des caractères précédents. On aimerait bien donner plus d'importance aux caractères qui portent plus d'information. Pour ça, on va rendre notre matrice de poids dépendante des données. On va, elle aussi, l'entraîner, pour retenir l'essentiel de l'information du contexte.

Pour ce faire, chaque token émettra deux vecteurs : un vecteur key, et un vecteur query. Le vecteur key décrira les informations qui constituent le token, le vecteur query décrira les informations dont le token a besoin. Avec de l'entraînement, le produit du vecteur query d'un caractère avec celui key d'un caractère avec aura une valeur élevée si leur affinité est importante.

Pour implémenter le principe de self-attention, on met en place des heads.



In [18]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) #(B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)
# wei donne, pour chaque entrée du batch, une pondération de l'information des caractères passés
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1) # Le softmax permet de normaliser notre pondération, en donnant de l'importance aux key(token_i).query(current_token)

v = value(x)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

Le principe d'attention est un méchanisme de communication qui peut être représenté par un graphe. Les différents sommets du graphe envoie des informations via leurs arcs. Dans notre cas, on a huit sommets alignés, (numérotés pour garder la notion d'espace qu'il n'y a pas dans l'attention) le premier est relié à tous les autres, le deuxième à tous sauf au premier etc (ils donnent l'information aux caractères suivants).

Le principe d'attention, c'est juste une manière d'indiquer qu'on définit un système de communication de l'information dans nos données d'entrée.

La self-attention, ça signifie simplement que les keys et les queries ont les mêmes sources (ici les tokens d'une entrée).

On va maintenant créer la classe Head que l'on utilisera pour mettre tout ça en ordre.

In [19]:
class Head(nn.Module) :
  def __init__(self, head_size) :
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

  def forward(self, x) :
    k = self.key(x)
    q = self.query(x)
    # Calcul des affinités entre les caractères
    wei = q @ k.transpose(-2, -1)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)

    v = self.value(x)
    out = wei @ v
    return out



Maintenant qu'on a notre classe Head, on va aller encore un peu plus loin.
On va s'intéresser aux MultiHeadAttention.

Ce sont, en gors, simplement des ensemble de Heads, et plutôt que de simplement appliquer une Head à une entrée x, on lui applique plein de heads.

In [20]:
class MultiHeadAttention(nn.Module) :
  def __init__(self, num_heads, head_size) :
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

  def forward(self, x) :
    return torch.cat([h(x) for h in self.heads], dim=-1)

On va ensuite avoir besoin d'un FeedForward, qui est simplement une couche linéaire de neurones suivis d'une activation non linéaire (ReLU).

Pourquoi a-t-on besoin de ce Feed Forward ?

Avec notre système Multi Heads pour implémenter la self attention, les tokens ont communiqué entre eux. Une fois qu'on l'a appliqué à l'entrée, les tokens ne sont plus seulement de simples tokens, ils portent l'information du contexte dans lequel ils apparaissent.

Maintenant, ce qu'il reste à faire semble évident : on a des informations bien optimisées, maintenant il va falloir choisir en se basant sur nos informations. C'est le rôle du Feed Forward.

Pour résumer :
1) On a nos entrées et leur positionnement.
2) On leur applique une self attention avec des multi heads pour optimiser le partage d'informations
3) On fait notre prédiction avec le feed forward

In [21]:
class FeedForward(nn.Module) :
  def __init__(self, n_embd) :
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd)
    )

  def forward(self, x) :
    return self.net(x)

Maintenant qu'on a défini nos simple Heads, nos Multi Heads, on peut s'en servir dans la définition de notre modèle.

In [22]:
class BigramLanguageModel(nn.Module):
  def __init__(self) :
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # matrice de taille vocab_size * n_embd
    self.position_embedding_table = nn.Embedding(block_size, n_embd) # matrice de taille block_size * n_embd
    self.sa_heads = MultiHeadAttention(4, n_embd//4)
    self.ffwd = FeedForward(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size) # matrice de taille n_embd * vocab_size, qui permettra de transformer token_embd en logits

  def forward(self, idx, targets=None) :
    B, T = idx.shape

    tok_emb = self.token_embedding_table(idx) # idx est de dimensions B * T, tok_emb est donc de dimensions B * T * C (où C = n_embd)
    pos_emb = self.position_embedding_table(torch.arange(T, device = device)) # dimensions T * C, ajoute l'information de la position du token dans le block
    x = tok_emb + pos_emb # dimensions B * T * C
    x = self.sa_heads(x) # On applique une tête de la self attention
    x = self.ffwd(x)
    logits = self.lm_head(x) # idx est de dimensions B * T, logits est donc de dimensions B * T * vocab_size

    if targets == None :
      loss = None
    else :
      B, T, C = logits.shape
      logits = logits.view(B*T, C) # on redimensionne nos prédictions pour calculer la loss
      targets = targets.view(B*T)

      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_tokens_generated) :
    for i in range(max_tokens_generated) :
      idx_cond = idx[:, -block_size:]  # tronquer à la longueur maximale autorisée
      logits, loss = self.forward(idx_cond)
      logits = logits[:, -1, :] # on isole les prédictions pour le prochain caractère basé sur celui qui précède
      probs = F.softmax(logits, dim=-1) # on calcule les probas
      idx_next = torch.multinomial(probs, num_samples=1) # on sélectionne le caractère suivant selon les probas obtenues
      idx = torch.cat((idx, idx_next), dim=1) # on ajoute le caractère suivant à notre entrée
    return idx

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

model = BigramLanguageModel()

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_tokens_generated=500)[0].tolist()))

step 0: train loss 4.1960, val loss 4.1948
step 300: train loss 2.3696, val loss 2.3766
step 600: train loss 2.2780, val loss 2.2883
step 900: train loss 2.2368, val loss 2.2521
step 1200: train loss 2.1875, val loss 2.1879
step 1500: train loss 2.1620, val loss 2.1630
step 1800: train loss 2.1441, val loss 2.1573
step 2100: train loss 2.1308, val loss 2.1383
step 2400: train loss 2.1154, val loss 2.1404
step 2700: train loss 2.1025, val loss 2.1138

IUC't I:
Sew civy? ou shok y lon nin b me dous, ois ton mal by orelot KETh hin ay vield sur in:
O:
CES:

asckist;
TELoon cibef thevepug tout mureasad,
NARDoredroon buperng rsergad,
Males, eithor d.
MCThere,
O bat ig, r:
I ath, thithineld fuefacul:
Whestre hind
F bemeelem.
RD:
Paill,
LEdud ak toof thoth y?
k,
CESCEs t ceishiolof bad inta o'thaind ne lenslare.
NGkenou st tistllay s.
BAfowhe h ld y fe in w tinolousish.
ENCARCHity BRTISLOBy'shuks n ingobetarde thtofooonoull pavishul uland ty s; he t
